In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.formula.api import ols
categorical_var = ["university", "sex", "gender_identity", "sexual_orientation", "ethnic_group", 'marital_status', 'student_accommodation', "work", 'RENDA_categorias', 'student_of', 'sedentary_behavior', "sedentary_2", "phq9", "phq9_severity", "gad7", "gad7_severity"]
db = pd.read_excel("BANCO_LIMPO_INT.xlsx")
#for var in categorical_var:
#    print(f"Response counts for {var}:")
#    print(db[var].value_counts())
#    print() 

creating contingency tables for every variable: 

In [44]:
xls = pd.ExcelFile('cat-tot-sheet.xlsx')
data_expanded = {}
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    expanded_df = df.loc[df.index.repeat(df['count'])].drop(columns=['count']).reset_index(drop=True)
    data_expanded[sheet_name] = expanded_df
with pd.ExcelWriter('contingency_tables.xlsx') as writer:
    for var1 in xls.sheet_names:
        for var2 in xls.sheet_names:
            if var1 != var2:
                merged = pd.concat([data_expanded[var1], data_expanded[var2]], axis=1)
                contingency_table = pd.crosstab(merged.iloc[:, 0], merged.iloc[:, 1], margins=True, margins_name='Total')
                contingency_table_df = contingency_table.reset_index()
                sheet_name = f'{var1}_vs_{var2}'[:30] 
                contingency_table_df.to_excel(writer, sheet_name=sheet_name, index=False)


In [73]:
with pd.ExcelWriter('crosstab_variables.xlsx') as writer:    
    for col1 in categorical_var:
        for col2 in categorical_var:
            if col1 != col2: 
                cont = pd.crosstab(db[col1], db[col2])
                cont.loc[''] = cont.sum()
                cont[''] = cont.sum(axis=1)
                cont = cont.reset_index()
                cont.rename(columns={col1: ''}, inplace=True)
                sheet_name = f'{col1}_vs_{col2}'[:30]
                cont.to_excel(writer, sheet_name=sheet_name, index=False)
                
        

testing for chi-squared and cramer's v:

In [74]:
xlsx_file = pd.ExcelFile('crosstab_variables.xlsx')
for sheet_name in xlsx_file.sheet_names:
    contingency_table = pd.read_excel(xlsx_file, sheet_name = sheet_name)
    contingency_table = contingency_table.fillna(0)
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
    n = contingency_table.to_numpy().sum()
    v_cramer = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))
    chi_squared_results = pd.DataFrame({
                    'Estatística': ['Chi-quadrado', 'valor p', 'Graus de liberdade', "V de Cramer"],
                    'Valor': [chi2, p, dof, v_cramer]
                })
    print(" ")
    print(sheet_name)
    print(chi_squared_results)
    print(" ")
    if p < 0.05: 
        print("Hipótese nula rejeitada; há dependência das variáveis") 
        if v_cramer < 0.3: print("Dependência fraca entre as variáveis; ")
        elif v_cramer > 0.6: print("Dependência forte entre variáveis")
        else: print("Dependência razoável entre variáveis")
    else: print("Hipótese alternativa rejeitada; variáveis independentes")


 
university_vs_sex
          Estatística         Valor
0        Chi-quadrado  2.958520e+02
1             valor p  1.408793e-42
2  Graus de liberdade  3.600000e+01
3         V de Cramer  6.501027e-02
 
Hipótese nula rejeitada; há dependência das variáveis
Dependência fraca entre as variáveis; 
 
university_vs_gender_identity
          Estatística         Valor
0        Chi-quadrado  2.693416e+02
1             valor p  1.427428e-32
2  Graus de liberdade  4.800000e+01
3         V de Cramer  5.392265e-02
 
Hipótese nula rejeitada; há dependência das variáveis
Dependência fraca entre as variáveis; 
 
university_vs_sexual_orientati
          Estatística         Valor
0        Chi-quadrado  3.619310e+02
1             valor p  3.176445e-40
2  Graus de liberdade  7.200000e+01
3         V de Cramer  5.095365e-02
 
Hipótese nula rejeitada; há dependência das variáveis
Dependência fraca entre as variáveis; 
 
university_vs_ethnic_group
          Estatística          Valor
0        Chi-quadrado   

In [76]:


# Load the Excel file
xlsx_file = pd.ExcelFile('crosstab_variables.xlsx')

for sheet_name in xlsx_file.sheet_names:
    # Read the contingency table and handle the index column
    contingency_table = pd.read_excel(xlsx_file, sheet_name=sheet_name)
    
    # Set the first column ('Unnamed: 0') as the index if it exists
    if 'Unnamed: 0' in contingency_table.columns:
        contingency_table.set_index('Unnamed: 0', inplace=True)

    # Replace NaN values with 0
    contingency_table = contingency_table.fillna(0)

    # Ensure that the contingency table contains only numeric data
    contingency_table = contingency_table.apply(pd.to_numeric)

    # Perform chi-squared test
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

    # Calculate the total number of observations
    n = contingency_table.to_numpy().sum()

    # Calculate Cramer's V
    v_cramer = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))

    # Prepare chi-squared results dataframe
    chi_squared_results = pd.DataFrame({
        'Estatística': ['Chi-quadrado', 'valor p', 'Graus de liberdade', "V de Cramer"],
        'Valor': [chi2, p, dof, v_cramer]
    })

    # Print results for each sheet
    print("\n")
    print(f"Sheet: {sheet_name}")
    print(chi_squared_results)
    print("\n")
    
    # Interpret results
    if p < 0.05:
        print("Hipótese nula rejeitada; há dependência das variáveis")
        if v_cramer < 0.3:
            print("Dependência fraca entre as variáveis")
        elif v_cramer > 0.6:
            print("Dependência forte entre as variáveis")
        else:
            print("Dependência razoável entre variáveis")
    else:
        print("Hipótese alternativa rejeitada; variáveis independentes")




Sheet: university_vs_sex
          Estatística         Valor
0        Chi-quadrado  7.378481e+01
1             valor p  5.758797e-07
2  Graus de liberdade  2.400000e+01
3         V de Cramer  3.982914e-02


Hipótese nula rejeitada; há dependência das variáveis
Dependência fraca entre as variáveis


Sheet: university_vs_gender_identity
          Estatística      Valor
0        Chi-quadrado  46.480922
1             valor p   0.113289
2  Graus de liberdade  36.000000
3         V de Cramer   0.025909


Hipótese alternativa rejeitada; variáveis independentes


Sheet: university_vs_sexual_orientati
          Estatística         Valor
0        Chi-quadrado  1.399867e+02
1             valor p  2.442434e-08
2  Graus de liberdade  6.000000e+01
3         V de Cramer  3.477176e-02


Hipótese nula rejeitada; há dependência das variáveis
Dependência fraca entre as variáveis


Sheet: university_vs_ethnic_group
          Estatística          Valor
0        Chi-quadrado   7.446215e+02
1             v